In [1]:
import netCDF4 as nc
from netCDF4 import Dataset
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# gsw oceanic toolbox: http://www.teos-10.org/pubs/Getting_Started.pdf
import gsw
from scipy.io import loadmat
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

In [2]:
eps_ds = "/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/data/nansen_legacy/GOS2020113_EPSI.nc"
hi = Dataset(eps_ds)
hhh = xr.open_dataset(eps_ds)

MissingDimensionsError: 'TIME' has more than 1-dimension and the same name as one of its dimensions ('SECTIONS', 'TIME'). xarray disallows such variables because they conflict with the coordinates used to label dimensions.

In [18]:
ctd_ds = "/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/data/nansen_legacy/GOS2020113_CTD.nc"
hallo = Dataset(ctd_ds)

In [22]:
result = pd.concat([hi, hallo], axis=0)

TypeError: cannot concatenate object of type '<class 'netCDF4._netCDF4.Dataset'>'; only Series and DataFrame objs are valid

In [15]:
import glob
GOS2020113_files = glob.glob("/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/data/nansen_legacy/NC_GOS2020113/*")

# create an empty list to store the datasets
datasets = []

# loop through each file and open as xarray dataset
for file_path in GOS2020113_files:
    dataset = xr.open_dataset(file_path)
    datasets.append(dataset)

In [17]:
datasets[2]

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*
Attributes: (12/87)
    platform_type:                      research vessel
    conventions:                        CF-1.6, ACDD-1.3, ATOMIX
    history:                            Version 1
    data_mode:                          D
    area:                               Arctic Ocean
    geospatial_lat_min:                 77.049
    ...                                 ...
    date_update:                        2023-02-13T22:28:00Z
    date_created:                       2023-02-13
    title:                              Ocean microstructure measurements usi...
    summary:                            Ocean microstructure profiles were co...
    comment:                            Profiles were obtained using an MSS (...
    citation:                           Fer, Ilker; Baumann, Till M.; Elliott...

In [ ]:
"/Users/Lisanne/Documents/AI4ER/Mres/ArcticTurbulence/data/nansen_legacy"